In [2]:
#!pip install "tensorflow_hub==0.4.0"
#!pip install --upgrade pip setuptools wheel
#!pip install -I tensorflow
#!pip install -I keras
#!pip install iterative-stratification
#!pip install -U sacremoses

    100% |████████████████████████████████| 1.4MB 22.6MB/s ta 0:00:01
    100% |████████████████████████████████| 583kB 34.8MB/s ta 0:00:01
fairing 0.5.3 has requirement tornado<6.0.0,>=5.1.1, but you'll have tornado 6.0.3 which is incompatible.
fairing 0.5.3 has requirement urllib3==1.24.2, but you'll have urllib3 1.24.1 which is incompatible.
datalab 1.1.4 has requirement six==1.10.0, but you'll have six 1.12.0 which is incompatible.
  Found existing installation: pip 19.0.3
    Uninstalling pip-19.0.3:
      Successfully uninstalled pip-19.0.3
  Found existing installation: setuptools 40.8.0
    Uninstalling setuptools-40.8.0:
      Successfully uninstalled setuptools-40.8.0
  Found existing installation: wheel 0.33.1
    Uninstalling wheel-0.33.1:
      Successfully uninstalled wheel-0.33.1
  Using cached https://files.pythonhosted.org/packages/f4/28/96efba1a516cdacc2e2d6d081f699c001d414cc8ca3250e6d59ae657eb2b/tensorflow-1.14.0-cp37-cp37m-manylinux1_x86_64.whl
  Using cached https:

  Using cached https://files.pythonhosted.org/packages/71/e3/19762fdfc62877ae9102edf6342d71b28fbfd9dea3d2f96a882ce099b03f/Keras_Applications-1.0.8-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/05/4b/55cfbfd3e5e85016eeef9f21c0ec809d978706a0d60b62cc28aeec8c792f/numpy-1.17.0-cp37-cp37m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/73/fb/00a976f728d0d1fecfe898238ce23f502a721c0ac0ecfedb80e0d88c64e9/six-1.12.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/28/6a/8c1f62c37212d9fc441a7e26736df51ce6f0e38455816445471f10da4f0a/Keras_Preprocessing-1.1.0-py2.py3-none-any.whl
     |████████████████████████████████| 25.2MB 47.7MB/s eta 0:00:01
  Created wheel for pyyaml: filename=PyYAML-5.1.2-cp37-cp37m-linux_x86_64.whl size=44104 sha256=c3cb996f3a1849dbcce29cdc8ab26f3aa89e6111b37adbf89a62a65b54931766
  Stored in directory: /home/jupyter/.cache/pip/wheels/d9/45/dd/65f0b38450c47cf7e5312883deb97d065e030c5cca0a365030
S

In [4]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, hamming_loss, roc_auc_score
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

mlb = MultiLabelBinarizer()

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.initializers import Constant

import tensorflow_hub as hub
import os
import re
from keras import backend as K
import keras.layers as layers
from keras.models import Model, load_model
from keras.engine import Layer

from sacremoses import MosesTokenizer

# Initialize session
#sess = tf.Session()
#K.set_session(sess)

/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passi

In [9]:
TEXT_DATA_DIR = f"../../datasets/cleanup_labelled.csv"

MAX_SEQUENCE_LENGTH = 1000
MAX_NUM_WORDS = 20000
EMBEDDING_DIM = 50
N_EPOCHS = 10
VALIDATION_SPLIT = 0.2

labels_index = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17]

In [6]:
class ElmoEmbeddingLayer(Layer):
    def __init__(self, **kwargs):
        self.dimensions = 1024
        self.trainable=True
        super(ElmoEmbeddingLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.elmo = hub.Module('https://tfhub.dev/google/elmo/2', trainable=self.trainable,
                               name="{}_module".format(self.name))
        

        self.trainable_weights += K.tf.trainable_variables(scope="^{}_module/.*".format(self.name))
        super(ElmoEmbeddingLayer, self).build(input_shape)

    def call(self, x, mask=None):
        result = self.elmo(K.squeeze(K.cast(x, tf.string), axis=1),
                      as_dict=True,
                      signature='default',
                      )['default']
        return result

    def compute_mask(self, inputs, mask=None):
        return K.not_equal(inputs, '--PAD--')

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.dimensions)

In [26]:
# Function to build model
def build_model(): 
    input_text = layers.Input(shape=(1,), dtype="string")
    embedding = ElmoEmbeddingLayer()(input_text)
    dense = layers.Dense(256, activation='relu')(embedding)
    pred = layers.Dense(17, activation='sigmoid')(dense)

    model = Model(inputs=[input_text], outputs=pred)

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()

    return model

In [13]:
# second, prepare text samples and their labels
print('Processing text dataset')
mt = MosesTokenizer(lang='en')
df = pd.read_csv(TEXT_DATA_DIR)
df.labels = df.labels.str.split('|').apply(lambda x: [int(i) for i in x])

print('Tokenizing data')
data = np.array([mt.tokenize(t, escape=False)[:150] for t in df.text])
mlb = MultiLabelBinarizer()
labels = mlb.fit_transform(df.labels)

# Cross-validation: split the data into a training set and a test set
cross_validation_sets = []
count = 0
mskf = MultilabelStratifiedKFold(n_splits=10, random_state=0)

for train_index, test_index in mskf.split(data, labels):
    count += 1
    print(f"Fold no. {count}")
    train_text, test_text = data[train_index], data[test_index]
    
    # Look into adapting the script to accept list of tokens instead
    train_text = [' '.join(t) for t in train_text]
    train_text = np.array(train_text, dtype=object)[:, np.newaxis]
    
    test_text = [' '.join(t) for t in test_text]
    test_text = np.array(test_text, dtype=object)[:, np.newaxis]
    
    train_label, test_label = labels[train_index], labels[test_index]
    cross_validation_sets.append((train_text, train_label, test_text, test_label))
print('Finished')

Processing text dataset
Tokenizing data
Fold no. 1
Fold no. 2
Fold no. 3
Fold no. 4
Fold no. 5
Fold no. 6
Fold no. 7
Fold no. 8
Fold no. 9
Fold no. 10
Finished


In [ ]:
count = 0
for train_text, train_label, test_text, test_label in cross_validation_sets:
    count += 1
    print(f"Fold {count}")
    
    model = build_model()
    model.fit(train_text, 
          train_label,
          epochs=N_EPOCHS,
          batch_size=32)
    model.save(f"elmo_{N_EPOCHS}_epochs_crossvalidated_fold_{count}.h5")

In [52]:
def metrics_avg(models_testx_testy, labels_, thres=0.3):
    def calc(model, test_x, test_y):
    predictions = model.predict(test_x)>thres
    metrics = classification_report(test_y, predictions, target_names=labels_, output_dict=True)
    metrics_df = pd.DataFrame.from_dict(metrics)
    h = hamming_loss(test_y, predictions)
    roc = roc_auc_score(test_y, predictions, average='micro')
        return metrics_df, h, roc

    model_1, test_x_1, test_y_1 = models_testx_testy[0]
    metrics_agg, ham, roc = calc(model_1, test_x_1, test_y_1)
    n = len(models_testx_testy)

    for model, test_x, test_y_1 in models_testx_testy[1:]:
        metrics, h, r = calc(model, test_x, test_y_1)
        metrics_agg += metrics
        ham += h
        roc += r

    return metrics_agg/n, ham/n, roc/n

In [59]:
labels = [str(i) for i in range(1,18)]
model = None
model = build_model()
model.load_weights('elmo_15epochs_crossvalidated_fold_1.h5')
#averaged_results = metrics_avg([(model, test_text, test_label)], labels)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_19 (InputLayer)        (None, 1)                 0         
_________________________________________________________________
elmo_embedding_layer_19 (Elm (None, 1024)              4         
_________________________________________________________________
dense_35 (Dense)             (None, 256)               262400    
_________________________________________________________________
dense_36 (Dense)             (None, 17)                4369      
Total params: 266,773
Trainable params: 266,773
Non-trainable params: 0
_________________________________________________________________


In [60]:
train_text, train_label, test_text, test_label =  cross_validation_sets[0]
avg_results = metrics_avg([(model, test_text, test_label)], labels)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


In [63]:
avg_results[2]

0.7789815538905387

In [65]:
model.fit(train_text, 
          train_label,
          epochs=5,
          batch_size=32)
model.save(f"elmo_20epochs_crossvalidated_fold_1.h5")

Epoch 1/5
4711/4711 [==============================] - 948s 201ms/step - loss: 0.0859 - acc: 0.9699
Epoch 2/5
4711/4711 [==============================] - 936s 199ms/step - loss: 0.0771 - acc: 0.9727
Epoch 3/5
4711/4711 [==============================] - 930s 197ms/step - loss: 0.0710 - acc: 0.9753
Epoch 4/5
4711/4711 [==============================] - 934s 198ms/step - loss: 0.0659 - acc: 0.9766
Epoch 5/5
4711/4711 [==============================] - 936s 199ms/step - loss: 0.0582 - acc: 0.9806


In [66]:
avg_results = metrics_avg([(model, test_text, test_label)], labels)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


In [69]:
avg_results[2]

0.7735558335056892